In [1]:
import sys
import os
sys.path.append('../')
os.chdir('../')

import pandas as pd
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime
import time
import threading
import json
import requests
import threading  
from src import utils

from bs4 import BeautifulSoup
import re

import spacy
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Posibles tesauros:

    - https://www.nlm.nih.gov/mesh/meshhome.html version en ingles (MeSH)
    - https://documentation.uts.nlm.nih.gov/rest/home.html

In [4]:
info = "data/prueba"

In [5]:
all_data = []
for element in os.listdir(info):
    nregistro = int(element[:-5])
    print(nregistro)
    data = utils.read_medicamento_info(os.path.join(info,element))
    all_data.append(data)
    print(data)

12723
[{'seccion': '1', 'titulo': 'NOMBRE DEL MEDICAMENTO', 'contenido': '<div>\r\n    <p style="margin:0pt; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-size:11pt">Antabus 250 mg comprimidos</span></p>\r\n  </div>', 'orden': 1}, {'seccion': '2', 'titulo': 'COMPOSICIÓN CUALITATIVA Y CUANTITATIVA', 'contenido': '<div>\r\n    <p style="margin:0pt"><span style="font-family:Times New Roman; font-size:11pt">Cada comprimido contiene 250 mg de disulfiram.</span></p>\r\n\r\n    <p style="margin:0pt"><span style="font-family:Times New Roman; font-size:11pt">&#xa0;</span></p>\r\n\r\n    <p style="margin:0pt"><span style="font-family:Times New Roman; font-size:11pt">Para consultar la lista completa de excipientes ver secci&#243;n 6.1.</span></p>\r\n\r\n    <p style="margin:0pt; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-size:11pt">&#xa0;</span></p>\r\n  </div>', 'orden': 1}, {'seccion': '3', 'titulo': 'FORMA FARMACÉUTICA', 

Lectura de todos los medicamentos que tenemos en la carpeta info, en este caso se enseña el nregistro del medicamento, 

In [11]:
for data in all_data:
    name_original = utils.extract_and_clean_text(data[0]["contenido"])
    print(name_original)
    name = utils.extract_text_before_number(name_original)
    text = utils.extract_and_clean_text(data[14]["contenido"])
    print(name_original,", ",name)

Antabus 250 mg comprimidos
Antabus 250 mg comprimidos ,  Antabus
Eutirox 25 microgramos comprimidos


Eutirox 50 microgramos comprimidos


Eutirox 75 microgramos comprimidos


Eutirox 88 microgramos comprimidos


Eutirox 100 microgramos comprimidos


Eutirox 112 microgramos comprimidos


Eutirox 125 microgramos comprimidos


Eutirox 137 microgramos comprimidos


Eutirox 150 microgramos comprimidos


Eutirox 175 microgramos comprimidos


Eutirox 200 microgramos comprimidos
Eutirox 25 microgramos comprimidos


Eutirox 50 microgramos comprimidos


Eutirox 75 microgramos comprimidos


Eutirox 88 microgramos comprimidos


Eutirox 100 microgramos comprimidos


Eutirox 112 microgramos comprimidos


Eutirox 125 microgramos comprimidos


Eutirox 137 microgramos comprimidos


Eutirox 150 microgramos comprimidos


Eutirox 175 microgramos comprimidos


Eutirox 200 microgramos comprimidos ,  Eutirox
ASPIRINA C 400 mg / 240 mg Comprimidos efervescentes
ASPIRINA C 400 mg / 240 mg Comprimidos efervesc

In [44]:
datos_antabus = all_data[0][14]
datos_eutirox = all_data[1][17]
datos_aspirina = all_data[2][15]

In [46]:
datos_aspirina

{'seccion': '4.8',
 'titulo': 'Reacciones adversas',
 'contenido': '<div>\r\n    <p style="margin:0pt; text-align:justify"><u><span style="font-family:Times New Roman; font-size:11pt;">&#193;cido acetilsalic&#237;lico:</span></u></p>\r\n\r\n    <p style="margin:0pt; text-align:justify"><span style="font-family:Times New Roman; font-size:11pt">&#xa0;</span></p>\r\n\r\n    <p style="margin:0pt 0pt 6pt; text-align:justify"><span style="font-family:Times New Roman; font-size:11pt">Los efectos adversos del &#225;cido acetilsalic&#237;lico son, en general, infrecuentes aunque importantes en algunos casos. En la mayor parte de los casos, los efectos adversos son una prolongaci&#243;n de la acci&#243;n farmacol&#243;gica y afectan principalmente al aparato digestivo. El 5-7% de los pacientes experimenta alg&#250;n tipo de efecto adverso.</span></p>\r\n\r\n    <p style="margin:0pt 0pt 0pt 35.45pt; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-size:11pt">&#xa0;

In [48]:
def bar(text):
    from IPython.display import display, HTML
    chart = HTML(text)
    # or chart = charts.plot(...)
    display(chart)
    return chart

chart = bar(datos_aspirina["contenido"])

Trastornos de la sangre y sistema linfático:,"Frecuentes (≥1/100 a <1/10): - aumento del riesgo de hemorragia, hemorragias asociadas al procedimiento, hematomas, epistaxis, hemorragias urogenitales, sangrado gingival, hipoprotrombinemia (con dosis altas). Poco frecuentes (≥1/1000 a <1/100): - anemia Raros (?1/10.000 a <1/1.000): - Las hemorragias pueden llevar a anemias hemorrágicas /deficiencia de hierro (debido a sangrados ocultos) que presentan los síntomas típicos, tales como astenia, palidez, hipoperfusión. Muy raros (< 1/10.000): - Hemorragia cerebral (especialmente en pacientes con hipertensión no controlada y que toman concomitantemente agentes antihemostáticos)."
"Trastornos respiratorios, torácicos y mediastínicos:","Frecuentes (≥1/100 a <1/10): - espasmo bronquial paroxístico, disnea grave, rinitis, enfermedad respiratoria exacerbada por el ácido acetilsalicílico, congestión nasal. Muy raros (< 1/10.000): - shock anafiláctico."
Trastornos gastrointestinales:,"Frecuentes (≥1/100 a <1/10): - úlcera gástrica, úlcera duodenal, hemorragia gastrointestinal (melenas, hematemesis), dolor abdominal y gastrointestinal, dispepsia, diarrea, náuseas, vómitos. Raros (≥ 1/10.000 a <1/1.000): - Inflamación gastrointestinal. Muy raros (< 1/10.000): - úlcera gastrointestinal con hemorragia y perforación."
Trastornos de la piel y tejido subcutáneo:,"Frecuentes (≥1/100 a <1/10): - urticaria, erupciones cutáneas, angioedema, prurito."
Trastornos generales y alteraciones en el lugar de administración:,"Poco frecuentes (≥1/1000 a <1/100): - síndrome de Reye (en menores de 16 años con procesos febriles, gripe o varicela. Ver sección 4.3)."
Trastornos hepatobiliares:,Poco frecuentes (≥1/1000 a <1/100): - hepatitis (particularmente en pacientes con artritis juvenil). Muy raros (< 1/10.000): - insuficiencia hepática transitoria con aumento de transaminasas del hígado.
Trastornos del sistema nervioso:,- mareos
Trastornos del oído y del laberinto:,"- tinnitus, sordera"
Trastornos renales y urinarios:,"- insuficiencia renal, lesión renal aguda y nefritis intersticial aguda"
Trastornos gastrointestinales:,- enfermedad diafragmática intestinal
Trastornos de la piel y tejido subcutáneo:,- sudoración


In [50]:
## Este metodo se encarga de quitar las etiquetas del html y eliminar tabs

# TODO:
#   - Mirar si funciona con todos los tipos de punto 4.8

html_content = datos_aspirina["contenido"]
# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Extract text and filter out empty lines
text = soup.get_text(separator='\n').strip()
cleaned_text = re.sub(r'\xa0', '', text)

lines = re.split(r'\n{2,}', cleaned_text)
for index,element in enumerate(lines):
    cleaned_text = re.sub(r'\n', '', element)
    lines[index] = cleaned_text

cleaned_text = '\n'.join(lines)

In [51]:
lines

['Ácido acetilsalicílico:',
 'Los efectos adversos del ácido acetilsalicílico son, en general, infrecuentes aunque importantes en algunos casos. En la mayor parte de los casos, los efectos adversos son una prolongación de la acción farmacológica y afectan principalmente al aparato digestivo. El 5-7% de los pacientes experimenta algún tipo de efecto adverso.',
 'Las reacciones adversas más características son:',
 'Trastornos de la sangre y sistema linfático:',
 'Frecuentes (≥1/100 a <1/10):',
 '- aumento del riesgo de hemorragia, hemorragias asociadas al procedimiento, hematomas, epistaxis, hemorragias urogenitales, sangrado gingival, hipoprotrombinemia (con dosis altas).',
 'Poco frecuentes (≥1/1000 a <1/100):',
 '- anemia',
 'Raros (?1/10.000 a <1/1.000):',
 '- Las hemorragias pueden llevar a anemias hemorrágicas /deficiencia de hierro (debido a sangrados ocultos) que presentan los síntomas típicos, tales como astenia, palidez, hipoperfusión.',
 'Muy raros (< 1/10.000):',
 '- Hemorrag

In [52]:
# Remove capital letters and periods
low_not_dots = ''.join([char for char in cleaned_text.lower() if char != '.'])
low_not_dots


'ácido acetilsalicílico:\nlos efectos adversos del ácido acetilsalicílico son, en general, infrecuentes aunque importantes en algunos casos en la mayor parte de los casos, los efectos adversos son una prolongación de la acción farmacológica y afectan principalmente al aparato digestivo el 5-7% de los pacientes experimenta algún tipo de efecto adverso\nlas reacciones adversas más características son:\ntrastornos de la sangre y sistema linfático:\nfrecuentes (≥1/100 a <1/10):\n- aumento del riesgo de hemorragia, hemorragias asociadas al procedimiento, hematomas, epistaxis, hemorragias urogenitales, sangrado gingival, hipoprotrombinemia (con dosis altas)\npoco frecuentes (≥1/1000 a <1/100):\n- anemia\nraros (?1/10000 a <1/1000):\n- las hemorragias pueden llevar a anemias hemorrágicas /deficiencia de hierro (debido a sangrados ocultos) que presentan los síntomas típicos, tales como astenia, palidez, hipoperfusión\nmuy raros (< 1/10000):\n- hemorragia cerebral (especialmente en pacientes co

In [53]:
a = low_not_dots.split("\n")

In [54]:
b = re.sub(':', '', low_not_dots)
c = b.split("\n")

In [14]:
import nltk
from nltk.tokenize import word_tokenize
# nltk.download('punkt')

# Supongamos que este es nuestro tesauro simplificado
tesauro = {
    "corazón": ["cardíaco", "miocardio"],
    "dolor": ["molestia", "incomodidad"]
}

# Función para buscar sinónimos en el tesauro
def buscar_sinonimos(palabra, tesauro):
    for key, sinonimos in tesauro.items():
        if palabra in sinonimos:
            return key
    return palabra

# Texto de ejemplo
texto = "El paciente experimenta molestia en el área cardíaca."

# Tokenización del texto
tokens = word_tokenize(texto)

# Reemplazo de sinónimos
texto_procesado = [buscar_sinonimos(palabra, tesauro) for palabra in tokens]

# Reconstruir el texto
texto_reconstruido = ' '.join(texto_procesado)
print("Texto Original:", texto)
print("Texto Procesado:", texto_reconstruido)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Texto Original: El paciente experimenta molestia en el área cardíaca.
Texto Procesado: El paciente experimenta dolor en el área cardíaca .


El texto a analizar debe ser procesado adecuadamente. Esto implica limpiar el texto (eliminar ruido, normalizar términos, etc.), 
tokenizarlo (dividirlo en palabras o frases individuales) y posiblemente lematizarlo (reducir las palabras a su forma base).

pip install nltk
pip install spacy
python -m spacy download es_core_news_sm # Descargar el modelo en español para spaCy

In [55]:

# Texto de ejemplo
texto = c

# Procesar cada línea del texto
new_texto = []
for linea in texto:
    linea_limpia = utils.limpiar_texto(linea)
    linea_lematizada = utils.lematizar_texto(linea_limpia)
    tokens = word_tokenize(linea_lematizada)
    new_texto.append(tokens)
    # print("Original:", linea)
    # print("Limpio:", linea_limpia)
    # print("Lematizado:", linea_lematizada)
    print("Tokens:", tokens)
    # print("\n")
# print(new_texto)

Tokens: ['ácido', 'acetilsalicílico']
Tokens: ['el', 'efecto', 'adverso', 'del', 'ácido', 'acetilsalicílico', 'ser', 'en', 'general', 'infrecuent', 'aunque', 'importante', 'en', 'alguno', 'caso', 'en', 'el', 'mayor', 'parte', 'de', 'el', 'caso', 'el', 'efecto', 'adverso', 'ser', 'uno', 'prolongación', 'de', 'el', 'acción', 'farmacológico', 'y', 'afectar', 'principalmente', 'al', 'aparato', 'digestivo', 'el', 'de', 'el', 'paciente', 'experimentar', 'alguno', 'tipo', 'de', 'efecto', 'adverso']
Tokens: ['el', 'reacción', 'adverso', 'más', 'característica', 'ser']
Tokens: ['trastorno', 'de', 'el', 'sangre', 'y', 'sistema', 'linfático']
Tokens: ['frecuente', 'a']
Tokens: ['aumento', 'del', 'riesgo', 'de', 'hemorragia', 'hemorragia', 'asociado', 'al', 'procedimiento', 'hematoma', 'epistaxis', 'hemorragia', 'urogenital', 'sangrado', 'gingival', 'hipoprotrombinemia', 'con', 'dosis', 'alto']
Tokens: ['poco', 'frecuente', 'a']
Tokens: ['anemia']
Tokens: ['raro', 'a']
Tokens: ['el', 'hemorragia',

In [56]:

listas_filtradas = [[elemento for elemento in lista if elemento not in stop_words] for lista in new_texto]


In [57]:
for element in listas_filtradas:
    print(element)

['ácido', 'acetilsalicílico']
['efecto', 'adverso', 'ácido', 'acetilsalicílico', 'ser', 'general', 'infrecuent', 'aunque', 'importante', 'alguno', 'caso', 'mayor', 'parte', 'caso', 'efecto', 'adverso', 'ser', 'prolongación', 'acción', 'farmacológico', 'afectar', 'principalmente', 'aparato', 'digestivo', 'paciente', 'experimentar', 'alguno', 'tipo', 'efecto', 'adverso']
['reacción', 'adverso', 'característica', 'ser']
['trastorno', 'sangre', 'sistema', 'linfático']
['frecuente']
['aumento', 'riesgo', 'hemorragia', 'hemorragia', 'asociado', 'procedimiento', 'hematoma', 'epistaxis', 'hemorragia', 'urogenital', 'sangrado', 'gingival', 'hipoprotrombinemia', 'dosis', 'alto']
['frecuente']
['anemia']
['raro']
['hemorragia', 'poder', 'llevar', 'anemia', 'hemorrágica', 'deficiencia', 'hierro', 'debido', 'sangrado', 'oculto', 'presentar', 'síntoma', 'típico', 'tal', 'astenia', 'palidez', 'hipoperfusión']
['raro']
['hemorragia', 'cerebral', 'especialmente', 'paciente', 'hipertensión', 'controlado

#### Todo el ejemplo entero.

In [8]:
info = "data/prueba"

nregistro = 12723
data = utils.read_medicamento_info(os.path.join(info,str(nregistro) + ".json"))

html_content = data[14]["contenido"]
# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Extract text and filter out empty lines
text = soup.get_text(separator='\n').strip()
cleaned_text = re.sub(r'\xa0', '', text)

lines = re.split(r'\n{2,}', cleaned_text)
for index,element in enumerate(lines):
    cleaned_text = re.sub(r'\n', '', element)
    lines[index] = cleaned_text

cleaned_text = '\n'.join(lines)

# Remove capital letters and periods
low_not_dots = ''.join([char for char in cleaned_text.lower() if char != '.'])
low_not_dots

a = low_not_dots.split("\n")

b = re.sub(':', '', low_not_dots)
c = b.split("\n")

texto = c
# Procesar cada línea del texto
new_texto = []
for linea in texto:
    linea_limpia = utils.limpiar_texto(linea)
    linea_lematizada = utils.lematizar_texto(linea_limpia)
    tokens = word_tokenize(linea_lematizada)
    new_texto.append(tokens)

listas_filtradas = [[elemento for elemento in lista if elemento not in stop_words] for lista in new_texto]


# TODO: Revisar usar space para quitar stopwords

In [9]:
listas_filtradas

[['disulfiram',
  'ser',
  'mismo',
  'sustancia',
  'relativamente',
  'tóxico',
  'observar',
  'escaso',
  'efecto',
  'secundario',
  'administrar',
  'sólo',
  'dosis',
  'terapéutico',
  'animal',
  'ser',
  'humano'],
 ['efecto',
  'secundario',
  'reacción',
  'disulfiramalcohol',
  'haber',
  'describir',
  'sección'],
 ['mayoría',
  'reacción',
  'disulfiram',
  'ser',
  'autolimitada',
  'representar',
  'riesgo',
  'vital',
  'paciente'],
 ['número',
  'reducido',
  'paciente',
  'poder',
  'experimentar',
  'adormecer',
  'leve',
  'transitorio',
  'fatigo',
  'impotencia',
  'cefalea',
  'erupción',
  'acneiform',
  'dermatitis',
  'alérgico',
  'sabor',
  'metálico',
  'ajo',
  'dos',
  'primero',
  'semana',
  'tratamiento',
  'síntoma',
  'soler',
  'desaparecer',
  'espontáneamente',
  'continuación',
  'tratamiento',
  'reducción',
  'posología'],
 ['reacción', 'adverso', 'ordenado', 'según', 'frecuencia', 'aparición'],
 ['trastorno', 'psiquiátrico'],
 ['rara', 'epis